# DB Tables:
 - User
 - Tokens
 - Tracks
 - playlists
 
![](https://github.com/spotify-song/DS/blob/master/references/DB_tables_illu.png?raw=true)


In [1]:
import sys
sys.path.insert(0, '../')
from spotify_users import UserData
from db import *

In [2]:
from os import getenv
from sqlalchemy import create_engine, update
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
import spotipy

load_dotenv()


engine = create_engine(getenv('DATABASE_URL'))
Session = sessionmaker(
                        autocommit=False,
                        autoflush=False,
                        bind=engine
                        )
base = declarative_base()

session = Session()

In [3]:
uri = getenv('uri') # must match in the Spotify app dashboard
SPOTIFY_CLIENT_ID = getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = getenv('SPOTIFY_CLIENT_SECRET')
# cache_path = ('../.user_cache')
scope = 'playlist-modify-public user-library-read user-top-read'

spot_cc = spotipy.oauth2.SpotifyOAuth(
                                    username='',
                                    client_id=SPOTIFY_CLIENT_ID,
                                    client_secret=SPOTIFY_CLIENT_SECRET,
#                                     cache_path=cache_path,
                                    scope=scope,
                                    redirect_uri=uri
                                    )
spot = spotipy.Spotify(auth_manager=spot_cc)

In [4]:
le_usero = spot.current_user()
le_usero

{'display_name': 'Agustinvargas',
 'external_urls': {'spotify': 'https://open.spotify.com/user/37t3cvb5u3o97hin4bsj40abw'},
 'followers': {'href': None, 'total': 1},
 'href': 'https://api.spotify.com/v1/users/37t3cvb5u3o97hin4bsj40abw',
 'id': '37t3cvb5u3o97hin4bsj40abw',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee85246625e42b2e0e895ea9449e',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:37t3cvb5u3o97hin4bsj40abw'}

In [21]:
le_usero['id']

'37t3cvb5u3o97hin4bsj40abw'

In [6]:
top_trackies = spot.current_user_top_tracks(limit=50, time_range='long_term')

In [7]:
top_tracks = spot.current_user_top_tracks(limit=50, time_range='long_term')
top_tracks_id = [top_tracks['items'][x]['id'] for x in range(len(top_tracks['items']))]
top_50_aud_feat = spot.audio_features(tracks=top_tracks_id)

In [8]:
for k, v in enumerate(top_50_aud_feat):
    print(v['id'])

0akyEssGRVHstqCSWXusJL
6plO0gM4tUvRC9TKFGIuaN
02gaYAEdeR6poHcBH1KUQF
0NeJjNlprGfZpeX2LQuN6c
7kWFRZdedr2gtfE8JDumVZ
5iSpfk6cDOSYePagAoG639
57mLRN6tfXwTRvp9oPWpop
54KsfVVnN4YWI2mMrnyUcC
1s3WD4gbNoEXHiuSTmAKaK
1jecO8NeYLsVWVptITz4c1
6KseaEAFSS63N2NPZtDnRL
4zFPUEMucYleIIUnYVoeZf
52N0IV8hLVkRmnpFclmCzK
1IF5UcqRO42D12vYwceOY6
71Mj2THXRicZhTFGzln3al
2UxrK7r4cyQOSh7wvdQTe1
17OkYffr0SdAcpcbwMkDDG
0JfsIu62NVXNQl2s7ATN37
0107Auhv91hE49iLoxtayt
4cJOLN346rtOty3UPACsao
51wUFdgpNsV8cVzu7i6N0l
6Zy0ITa16EjCAbbGuPzdRi
7v3YlquaNhK2GYKzxovSEp
4RrOSjdnV8rkpIuOIfkKYS
1sJev5Y7VI2Ke8AwUpnh0l
3PUbNbybe6dTMWdUt9vQ02
2TzkIzgzIHhewMxyh1u4hh
325e3Lkx4AodxGNsqHiaG4
6fcS6fncRVP8rldHjriZHS
1yTJg3lyUPmwbnve82twH5
3QIQtCPni57ZcSPzz7JDxt
5ONAA8z6SvFBniu8zXz1Ax
7ukRl9q1yVYO2j5SXwvjaB
4utNmwmHfwvzPBT3kMtwEN
0xSqHQ5wv80hNkpU50vPc7
3PVgZvHQdNJHR0ZLN1u8vn
1VA0QtG2DXGF4k6fYz70PE
2nU1HJ8GcMPkatnvI6KjP4
4RNYL9drYkmWYpDyfknta9
58iNllszkXpDOcYRgcfLfH
4Mkpse99DB8TNijNjPpCbn
4m6rvm52fAgZk7ffqgevcV
3iRppHTSuWMVJVMdMD9knF
7KX65PC1UZu

In [9]:
# get's user playlists
playlists = spot.user_playlists('37t3cvb5u3o97hin4bsj40abw')
playlist_ids = []
for playlist_item in playlists['items']:
    playlist_ids.append(playlist_item['id'])

playlist_ids

['2Pk3OLPiVXIZHieWs5ZGHp',
 '5xT24CFEoMM2udviqtc3Pb',
 '3CzEMPwUNr3Ci2U1C8nvM6',
 '3BwjT1CuslwpMqNnPK4mYe',
 '2sSerqYIlV8MbGdACTnFTG',
 '2J33VEAP3HnxACz0ntmKau']

In [10]:
# get's playlist tracks
all_tracks_ids = []
for playlist_id in playlist_ids:
    playlist_trx = spot.playlist_tracks(playlist_id, offset=1, fields='items.track.id')
    for trx in playlist_trx['items']:
        all_tracks_ids.append(trx['track']['id'])

In [11]:
len(all_tracks_ids)

159

### Code for `UserData.get_audio_features()` method

In [28]:
# takes all saved tracks for a given user
lst = []
offset = 0
result = spot.current_user_saved_tracks(offset=offset)
while result['next'] != None:
    result = spot.current_user_saved_tracks(offset=offset)
    lst.append(result['items'])
    offset += 20

new_lst = []
for val in lst:
    for trk in val:
        new_lst.append(trk['track']['id'])

In [29]:
# getting track id from result
result['items'][1]['track']['id'], lst[1][19]['track']['id']


('52N0IV8hLVkRmnpFclmCzK', '325e3Lkx4AodxGNsqHiaG4')

### Code for `UserData.get_audio_features()` method

In [31]:
new_lst = []
for val in lst:
    for trk in val:
        new_lst.append(trk['track']['id'])

In [18]:
# runs through the list of track IDs and generates the a list of audio features
# for each of the songs

offset = 50
onset = 0
audio_feat = []
while offset:
    print(onset, offset)
    if offset >= len(new_lst):
        audio_feat.append(spot.audio_features(tracks=new_lst[onset:]))
        break
    audio_feat.append(spot.audio_features(tracks=new_lst[onset:offset]))
    onset += 50
    offset += 50

0 50
50 100
100 150


In [54]:
# Condenses the audio_features to a single list

lster = []
for i in audio_feat:
    for j in i:
        lster.append(j)
len(lster), len(new_lst)

(134, 134)

In [17]:
# testing the get_audio_features method in the UserData class 💃 

test = UserData()
audio_fe = test.get_audio_features(track_ids=new_lst, spot_session=spot)

In [24]:
playlist_trks = test.get_playlists_trx(spot_session=spot, user_id=le_usero['id'])